In [2]:
import re
import unicodedata
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

# Cargar el dataset
data = pd.read_csv('test.csv')

# Función para limpiar los subtítulos eliminando metadatos y normalizando los caracteres
def clean_subtitles(text):
    text = re.sub(r'WEBVTT.*\n', '', text)
    text = re.sub(r'Kind:.*\n', '', text)
    text = re.sub(r'Language:.*\n', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\d{2}:\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}:\d{2}.\d{3}', '', text)
    
    # Reemplazar tildes y mantener la ñ
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    
    # Convertimos letras acentuadas a letras normales (a, e, i, o, u)
    text = re.sub(r'á', 'a', text)
    text = re.sub(r'é', 'e', text)
    text = re.sub(r'í', 'i', text)
    text = re.sub(r'ó', 'o', text)
    text = re.sub(r'ú', 'u', text)
    
    # Eliminar espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Aplicar la limpieza al dataset
data['Cleaned_Subtitles'] = data['Subtitles'].apply(clean_subtitles)

In [7]:
# Definir los patrones de eventos clave utilizando expresiones regulares
event_patterns = {
    'Goles': r'\b(gol|anotacion|marco|marcaron)\b',
    'Tarjetas Amarillas': r'\b(tarjeta amarilla|amonestacion|amarilla)\b',
    'Tarjetas Rojas': r'\b(tarjeta roja|expulsado|expulsion|expulsar|roja|expulsaron)\b',
    'Tiros de Esquina': r'\b(corner|tiro de esquina)\b',
    'Saques de Banda': r'\b(saque de banda)\b',
    'Faltas': r'\b(falta|infraccion|foul|faul)\b',
    'Penales': r'\b(penal|penalty)\b',
    'Posesión de balón': r'\b(posesion)\b',
    'Fueras de juego': r'\b(fuera de juego|offside)\b',
    'Tiros': r'\b(tiro|disparo)\b'
}

# Función para etiquetar un fragmento de texto con el evento correspondiente
def label_events(text):
    labels = []
    for event, pattern in event_patterns.items():
        if re.search(pattern, text, re.IGNORECASE):
            labels.append(event)
    return labels

# Aplicar la función para generar etiquetas
data['Event_Labels'] = data['Cleaned_Subtitles'].apply(label_events)

# Convertir las listas de eventos en una única etiqueta simplificada por fragmento
def simplify_labels(event_list):
    if 'Goles' in event_list:
        return 0  # Goles
    elif 'Tarjetas Amarillas' in event_list:
        return 1  # Tarjeta Amarilla
    elif 'Tarjetas Rojas' in event_list:
        return 2  # Tarjeta Roja
    elif 'Tiros de Esquina' in event_list:
        return 3  # Tiros de Esquina
    elif 'Faltas' in event_list:
        return 4  # Faltas
    elif 'Penales' in event_list:
        return 5  # Penales
    else:
        return 6  # Ningún evento

data['Simplified_Labels'] = data['Event_Labels'].apply(simplify_labels)

# Función para detectar eventos en un fragmento de texto
def detect_events(text):
    event_counts = {event: 0 for event in event_patterns}  # Inicializamos el conteo de eventos
    for event, pattern in event_patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        event_counts[event] += len(matches)  # Contamos cuántas veces aparece cada evento
    return event_counts

In [8]:
# Dividimos los datos en entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Aumentamos el tamaño máximo de secuencias para capturar más contexto
maxlen = 200  # Aumentamos el tamaño máximo de la secuencia para capturar más contexto

# Tokenización del texto
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data['Cleaned_Subtitles'])

train_sequences = tokenizer.texts_to_sequences(train_data['Cleaned_Subtitles'])
test_sequences = tokenizer.texts_to_sequences(test_data['Cleaned_Subtitles'])

# Rellenamos las secuencias para que tengan la misma longitud
X_train = pad_sequences(train_sequences, maxlen=maxlen)
X_test = pad_sequences(test_sequences, maxlen=maxlen)

# Convertimos las etiquetas en arrays
y_train = train_data['Simplified_Labels']
y_test = test_data['Simplified_Labels']

# Definimos el modelo LSTM con dropout
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=maxlen),
    tf.keras.layers.LSTM(256, return_sequences=False),
    tf.keras.layers.Dropout(0.3),  # Añadimos dropout después de la LSTM
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Añadimos dropout antes de la capa final
    tf.keras.layers.Dense(7, activation='softmax')  # 7 clases de eventos
])

# Compilamos el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Definimos EarlyStopping para detener el entrenamiento cuando el modelo deja de mejorar
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entrenamos el modelo con más épocas y early stopping
history = model.fit(X_train, y_train, 
                    epochs=100,  # Aumentamos las épocas
                    validation_data=(X_test, y_test), 
                    batch_size=32, 
                    callbacks=[early_stopping])

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 231ms/step - accuracy: 0.5922 - loss: 1.9287 - val_accuracy: 1.0000 - val_loss: 1.8231
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9758 - loss: 1.7317 - val_accuracy: 1.0000 - val_loss: 0.0829
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9797 - loss: 0.2041 - val_accuracy: 1.0000 - val_loss: 2.9818e-04
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.9875 - loss: 0.1114 - val_accuracy: 1.0000 - val_loss: 3.4940e-04
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9836 - loss: 0.1324 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9758 - loss: 0.1353 - val_accuracy: 1.0000 - val_loss: 0.0662


In [9]:
# Función para convertir predicciones de modelos en eventos legibles
def prediction_to_event(pred):
    event_index = np.argmax(pred)
    event_mapping = {0: 'Goles', 1: 'Tarjetas Amarillas', 2: 'Tarjetas Rojas', 3: 'Tiros de Esquina', 4: 'Faltas', 5: 'Penales', 6: 'Ningún evento'}
    return event_mapping[event_index]

# Aplicamos el modelo a nuevas narraciones
predictions = model.predict(X_test)

# Función mejorada para asignar los eventos detectados a Equipo 1 y Equipo 2
def assign_events_to_teams(text, team1_name='Equipo 1', team2_name='Equipo 2'):
    team1_stats = {event: 0 for event in event_patterns}
    team2_stats = {event: 0 for event in event_patterns}
    
    # Procesamos el texto en oraciones o fragmentos
    sentences = text.split('.')
    
    # Alternamos entre los equipos para asignar los eventos
    current_team = 1
    
    for sentence in sentences:
        event_counts = detect_events(sentence)
        
        # Asignar los eventos al equipo correspondiente
        if current_team == 1:
            for event, count in event_counts.items():
                team1_stats[event] += count
            current_team = 2  # Alternamos al Equipo 2
        else:
            for event, count in event_counts.items():
                team2_stats[event] += count
            current_team = 1  # Volvemos al Equipo 1
    
    return team1_stats, team2_stats

# Generar estadísticas para las narraciones de prueba
for i in range(5):  # Revisamos las primeras 5 narraciones para verificar las predicciones y las estadísticas
    print(f"Narración: {test_data['Cleaned_Subtitles'].iloc[i]}")
    print(f"Predicción del evento: {prediction_to_event(predictions[i])}")
    team1_stats, team2_stats = assign_events_to_teams(test_data['Cleaned_Subtitles'].iloc[i])
    print(f'Estadísticas Equipo 1: {team1_stats}')
    print(f'Estadísticas Equipo 2: {team2_stats}')
    print('---')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Narración: WEBVTT Kind: captions Language: es hola queridos televidentes canal de futbol NTV plus invito Bienvenidos a Turin para el super partido y la ronda 29. campeonato italiano Super partido del partido de la Juventus Roma para Esta gira esta por cerrar y esto probablemente sea un juego. y campeon italiano si y la Roma ganara la diferencia llegara a 9 puntos de la roma juventus Y y 7 puntos para la Lazio Roma asi unidos tras derrotar a un talento con Con un resultado de 3 a 0, la Lazio pasa a ser segunda ya dos puntos por delante de la Juventus La Juventus sigue siendo 3 pero este partido y este Fabio Capello Roma entrenador Carlo Ancelotti llegara a la Juventus y a la plantilla Roma en la puerta de la nueva 15 Sabine 19 cm y 6 l a l 2 cafe 17 tommasi 4 nota a Christiana tambien 32 candelas 10 . 18 batistuta 24 delvig al capitan del equipo tata Van der Sar defiende la porteria 5 de la Juventus alla 12 Juliana 4 Montero 7 523 zambotto 20 nardi

In [1]:
team1_stats

NameError: name 'team1_stats' is not defined

In [2]:
team2_stats

NameError: name 'team2_stats' is not defined